# Frontier Models

This section uses AWS Bedrock to run inference tests using the Claude and Open AI models. 

In [31]:
%package install boto3=1.34.51 langchain_aws

Running: micromamba install boto3=1.34.51 langchain_aws --yes --quiet --log-level=error
error    libmamba Could not solve for environment specs
    The following package could not be installed
    └─ langchain_aws does not exist (perhaps a typo or a missing channel).
critical libmamba Could not solve for environment specs

Note: Packages not from Bloomberg channels are not vetted by Bloomberg.
Please restart the Jupyter kernel if you run into any issues after installing or updating packages via %package.



In [29]:
import json
import boto3
import botocore
import instructor

from prompts import SYSTEM_PROMPTS

In [2]:
# boto3 must be version 1.34.51 or higher
boto3.__version__

'1.34.51'

In [3]:
boto3_bedrock = boto3.client('bedrock-runtime')

In [20]:
model = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

In [5]:
prompt = f'{SYSTEM_PROMPTS['']}'

In [25]:
# Claude request
doc_message = {
    "role": "user",
    "content": [
        { "text": prompt }  # We add our prompt here 
    ]
}

native_request = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "temperature": 0.5,
    "messages": [
        # {
        #     "role":"system",
        #     "content": [{"type":"text", "text":"you are a helpful chatbot"}]
        # },
        {
            "role":"user",
            "content": [{"type":"text", "text":prompt}]
        }
    ]
}

body = json.dumps(native_request)

In [26]:
GUARDRAIL_ID = os.environ['BEDROCK_GUARDRAIL_ID']
GUARDRAIL_VERSION = os.environ['BEDROCK_GUARDRAIL_VERSION']

modelId = model
accept = 'application/json'
contentType = 'application/json'

try:
    
    response = boto3_bedrock.invoke_model(
        body=body, 
        modelId=modelId, 
        accept=accept, 
        contentType=contentType,
        #guardrailIdentifier = GUARDRAIL_ID, 
        #guardrailVersion = GUARDRAIL_VERSION, 
        trace = "ENABLED"
    )
    response_body = json.loads(response.get('body').read())
    #print(response_body.get('generation')) #Llama response template
    print(response_body.get('content')[0].get('text')) #Anthropic response template

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

The capital of France is Paris. It's not only the capital city but also the largest city in France, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.


In [27]:
response_body

{'id': 'msg_bdrk_01TB4f8JesY5R93mbfCEkPYy',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-7-sonnet-20250219',
 'content': [{'type': 'text',
   'text': "The capital of France is Paris. It's not only the capital city but also the largest city in France, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral."}],
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 14,
  'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'output_tokens': 51}}

In [12]:
GUARDRAIL_VERSION

'1'

In [30]:
from langchain_aws import ChatBedrockConverse

ModuleNotFoundError: No module named 'langchain_aws'